In [67]:
import os
# Change to your own
PATH_NAME = 'C:/Users/woobi/Documents/LearningFoodPreferences2/data/'
PATH_NAME2 = 'C:/Users/woobi/Documents/LearningFoodPreferences2/'


In [68]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\woobi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [69]:
import pandas as pd
import os
from nltk.stem import WordNetLemmatizer
import numpy as np
from gensim.models import KeyedVectors
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict
import pickle
import math
import re
from collections import Counter

In [70]:
# Load word2vec model
model = KeyedVectors.load(PATH_NAME + "한국어_음식모델_한상민.kv", mmap='r')
#path = PATH_NAME2 + "sample_food_logs/sample_food_log.csv"

with open(PATH_NAME + 'wweia_synonym_cats.pickle', 'rb') as handle:
  wweia_synonym_cats = pickle.load(handle)

In [71]:
# from gensim.models import KeyedVectors
# import csv

# # Load Word2Vec model
# model = KeyedVectors.load(PATH_NAME + "한국어_음식모델_한상민.kv", mmap='r')

# # Open a CSV file for writing
# csv_file_path = "word_embeddings.csv"
# with open(csv_file_path, 'w', newline='', encoding='utf-8') as csvfile:
#     # Create a CSV writer object
#     csv_writer = csv.writer(csvfile)
    
#     # Iterate through each word in the vocabulary
#     for word_index in range(len(model.index_to_key)):
#         word = model.index_to_key[word_index]
        
#         # Get the word embedding
#         embedding = model[word]
        
#         # Write the word and its embedding to the CSV file
#         csv_writer.writerow(list(embedding))


In [101]:
# Getting a sample of what an embedding looks like
model["레드컵"]

KeyError: "Key '레드컵' not present"

In [73]:
# Load WWEIA food categories csv. Contains the food category codes and associated verbal food categories.
# The "same_category" column lists which categories are considered synonymous.
wweia_food_categories = pd.read_csv(PATH_NAME + 'wweia_food_categories_addtl.csv')
wweia_food_categories

,NO,식품명,식품대분류,식품상세분류
0,1,꿩불고기,구이류,육류구이
1,2,닭갈비,구이류,육류구이
2,3,닭갈비,구이류,육류구이
3,4,닭꼬치,구이류,육류구이
4,5,더덕구이,구이류,채소류구이
...,...,...,...,...
7678,7679,<아내의 식탁>짜.수.치,볶음류,떡볶이류
7679,7680,<아내의 식탁>바질베스토감바스,볶음류,어패류볶음
7680,7681,<아내의 식탁>밀푀유스키야키,찌개 및 전골류,채소류찌개.전골
7681,7682,<파빌리온>양갈비 스테이크,구이류,육류구이


In [74]:
# Load the vocabulary of WWEIA food descriptions

food_words = pd.read_csv(PATH_NAME + 'food_words.csv')
food_words = food_words.iloc[:,1]
food_words = food_words.to_list()
food_words

['메밀전병',
 '수수부꾸미',
 '약식',
 '경단(깨)',
 '경단(콩)',
 '기피편',
 '녹두시루떡',
 '무지개떡',
 '송편(깨)',
 '송편(콩)',
 '수수팥떡',
 '시루떡',
 '쑥떡',
 '절편',
 '찹쌀떡',
 '마떡',
 '치즈마떡',
 '가래떡',
 '깨송편',
 '꿀떡',
 '모듬찰떡',
 '백설기',
 '쑥절편',
 '인절미',
 '증편',
 '참깨경단',
 '팥시루떡',
 '팥찹쌀떡',
 '제주도 빙떡 & 귤간장소스',
 '코코스페셜호두과자',
 '선물용 대',
 '선물용 중',
 '선물용 소',
 '미니케이스 럭셔리',
 '미니케이스 대',
 '미니케이스 소',
 '레드컵',
 '초코컵',
 '프리백',
 '간식용11개',
 '간식용19개',
 '간식용38개',
 '단체간식용 호두과자',
 '망고 마카롱',
 '초코 마카롱',
 '딸기 마카롱',
 '피스타치오 마카롱',
 '카라멜 마카롱',
 '유자 레몬 마카롱',
 '마다가스카르 바닐라 마카롱',
 '초코롱 프랄리네',
 '초코롱 바닐라',
 '초코롱 피스타치오',
 '초코롱 오렌지',
 '초코롱 산딸기',
 '초코롱 바나나',
 '산딸기 마카롱',
 '로즈베리 생크림',
 '그린티 마카롱',
 '벨지안 초콜릿 마카롱',
 '블루베리 마카롱',
 '딸기 마카롱',
 '바닐라 마카롱',
 '초코 마카롱',
 '블루베리 마카롱',
 '뛰일 아몬드',
 '마카롱',
 '다크 초콜릿 마카롱',
 '바닐라 마카롱',
 '블루베리 마카롱',
 '스트로베리 마카롱',
 '우도 땅콩 다쿠아즈',
 '제주 녹차 다쿠아즈',
 '카라멜 스틱',
 '이디야키즈 유기농 곡물바 딸기',
 '미니 코코 크런치',
 '얼그레이 초코 마카롱',
 '바닐라 빈 마카롱',
 '솔티카라멜 마카롱',
 '라즈베리 마카롱',
 '솔티카라멜 다쿠아즈',
 '초코바나나 다쿠아즈',
 '황금 보리 초코 건빵',
 '아이스 크로칸 슈',
 '오후의 마카롱모카',
 '마카롱모카파티',
 '화과자명가(대)',
 '바삭

In [75]:
# Load wweia_data, which contains the FNDDS foods, their nutritional information, and their food category

wweia_data = pd.read_csv(PATH_NAME + 'wweia_data.csv')
wweia_data

,식품코드,식품명,NO,식품대분류,식품상세분류,1회제공량,내용량_단위,에너지(㎉),수분(g),단백질(g),...,총 식이섬유(g),칼슘(㎎),철(㎎),인(㎎),셀레늄(㎍),나트륨(㎎),비타민 A(㎍ RE),비타민 B1(㎎),비타민 B2(㎎),비타민 C(㎎)
0,D000117,메밀전병,1,곡류 및 서류,곡류 및 서류,100,g,167.83,64.57,6.18,...,3.53,35.64,1.19,87.29,6.44,345.8,0,0.12,0.03,1.71
1,D000123,수수부꾸미,2,곡류 및 서류,곡류 및 서류,100,g,247.89,44.53,5.73,...,7.95,17.24,1.31,111.36,2.59,261,0,0.12,0.03,0
2,D018006,약식,3,곡류 및 서류,곡류 및 서류,100,g,219.87,47.2,4.21,...,2.63,13.38,-,43.44,3.24,221.18,0,0,0.01,0
3,D000112,경단(깨),4,곡류 및 서류,떡류,100,g,319.47,19.89,4.44,...,5.44,62.25,0.87,65.9,6.66,180,0,0,0.01,0
4,D000113,경단(콩),5,곡류 및 서류,떡류,100,g,314.32,21.25,3.51,...,2.43,10.41,0.53,44.16,3.39,225.83,0,0,0.01,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12314,D018800,간장치킨 (순살),12315,튀김류,치킨류,100,g,-,-,17,...,-,-,-,-,-,450,-,-,-,-
12315,D000481,대구포,12316,포류,포류,100,g,140.84,56.38,31.97,...,1.18,154.63,0.39,288.6,59.27,3875.95,0,0.07,0.07,0
12316,D000074,물회,12317,회류,회류,700,g,529.36,583.5,34.4,...,15.1,212.11,1.88,463.43,77.71,2095.9,7.96,0.42,0.22,3.04
12317,D000075,생선물회,12318,회류,회류,800,g,577.97,660.9,34.3,...,11.2,185.95,2.58,474.27,100.38,2779.74,13.89,0.16,0.2,16.04


In [76]:
# Load embeddings
wweia_embeddings = np.loadtxt(PATH_NAME + 'word_embeddings.csv', delimiter = ",")
wweia_embeddings.shape

(3710, 100)

In [77]:
def get_most_pop_categories(curr_log, n, col_name):

  cats = curr_log.loc[:, col_name]

  cats_counter = Counter(cats)
  counter_results = cats_counter.most_common(n)
  
  counter_list = [elem[0] for elem in counter_results]
  
  return counter_list

In [78]:
# initialize lemmatizer
lemmatizer = WordNetLemmatizer()

In [79]:
unhelpful_words = ["with", "or", "cooked", "to", "and", "as", "ns", "cooking", "in", "made", "from", " ", "canned", "frozen", "type", "eaten", "of", "on", "fresh", "nfs", "ready", "strained", "style", "than", "prepared", "method", "stewed", "drained", "homemade", "home"]

In [80]:
# Only keep words in phrase that are in food_words

def reduce_with_food_words(comma_phrase):
  comma_phrase_reduced = [lemmatizer.lemmatize(word.lower()) for word in comma_phrase.split() if (lemmatizer.lemmatize(word.lower()) in food_words or word.lower() in food_words) and (lemmatizer.lemmatize(word.lower()) not in unhelpful_words or word.lower() not in unhelpful_words)]
  return comma_phrase_reduced

In [81]:
# Format of food names seems to be (Optional) Company, Food, Details about food
# run function to guess if first or second comma phrase has Food

def process_food_log(curr_log, wweia_synonym_cats):
  curr_log['predicted_categories_number'] = 0
  curr_log['predicted_categories_words'] = ""
  curr_log['max_cosim_score'] = 0
  curr_log['most_sim_food'] = ""
  curr_log['reciprocal_rank'] = 0.0
  curr_log['sym_reciprocal_rank'] = 0.0

  for i in range(curr_log.shape[0]):
    descrip = curr_log.loc[i, 'Food Name']
    descrip_split = descrip.split(", ")
    # Reduce food log description to words in food_words
    first_phrase_num_words = reduce_with_food_words(descrip_split[0])
    pre_embedding = []

    if len(descrip_split) > 1:
      second_phrase_num_words = reduce_with_food_words(descrip_split[1])
      descrip_split_0_words = descrip_split[0].split()
      descrip_split_1_words = descrip_split[1].split()

      begin_first = False if len(second_phrase_num_words)/len(descrip_split_1_words) > len(first_phrase_num_words)/len(descrip_split_0_words) else True
      
      if begin_first:
        last_phrase = descrip

      if begin_first == False:
        three_phrases = descrip.partition(", ")
        last_phrase = three_phrases[2]

    else:
      last_phrase = descrip
    last_phrase_comma = last_phrase.split(", ")
    for j in range(len(last_phrase_comma)):
      split_comma = last_phrase_comma[j].split(" ")
      for elem in split_comma:
        elem = elem.lower()
        if elem not in unhelpful_words:
          pre_embedding.append(elem)

    word_embed = np.zeros(shape = (1, len(model["불고기"])))
    if len(pre_embedding) > 0:
      # Turned edited_descrip into embedding vector by averaging the words
      word_embed = np.zeros(shape = (1, len(model["불고기"])))
      num_words = 0
      for word in pre_embedding:
        word = word.lower()
        if word in model:
          num_words += 1
          word_embed += model[word]

      if num_words != 0:
        word_embed /= num_words
    # Compare to the other vectors
    similarities = cosine_similarity(word_embed, wweia_embeddings) # (1, 7918) for each wweia food
    # Finding WWEIA foods with highest similarity with each
    # food log food. Row has sorted list of WWEIA food IDs 
    # for each food log food.
    to_keep = np.sort(similarities, axis=1)
    to_keep_args = np.argsort(similarities, axis=1)
    indices = np.flip(to_keep_args, axis = 1)
    
    sym_rank = 1000000
    # record RR here
    for index in range(indices.shape[1]):
      true_cat = curr_log.loc[i, 'wweia_food_category_code']
      if math.isnan(true_cat): continue
      if wweia_data.loc[indices[0,index], 'NO'] == true_cat:
        rank = index
        rr = 1 / (rank+1)
        if sym_rank > rank:
          sym_rank = rank
          sym_rr = 1 / (rank+1)
        break
      else:
        
        if wweia_data.loc[indices[0,index], 'NO'] in wweia_synonym_cats[true_cat]:
          if sym_rank > index:
            sym_rank = index
            sym_rr = 1 / (index+1)
    most_sim_food_index = indices[0,0]

    most_sim_food_row = wweia_data.iloc[most_sim_food_index, :]
    highest_cat_num = most_sim_food_row['NO']
    highest_cat_words = wweia_food_categories.loc[wweia_food_categories['NO'] == highest_cat_num, '식품명']
    curr_log.loc[i, 'predicted_categories_number'] = highest_cat_num
    curr_log.loc[i, 'predicted_categories_words'] = highest_cat_words.to_list()[0]
    curr_log.loc[i, 'max_cosim_score'] = np.array2string(to_keep[0,-5:])
    curr_log.loc[i, 'most_sim_food'] = most_sim_food_row['식품명']
    curr_log.loc[i, 'reciprocal_rank'] = rr
    curr_log.loc[i, 'sym_reciprocal_rank'] = sym_rr
  return curr_log

In [82]:
import pandas as pd

# CSV 파일 읽기
df = pd.read_csv('C:/Users/woobi/Documents/LearningFoodPreferences2/data/wweia_food_categories_addtl.csv')

# 식품대분류 및 상품번호 모으기
category_no_lists = df.groupby('식품대분류')['NO'].apply(list)

# 출력을 위한 딕셔너리 생성
output_dict = {}

# 해당 형식에 맞게 딕셔너리 채우기
for i, (category, no_list) in enumerate(category_no_lists.items(), 1):
    output = []
    no_list.sort()
    start_no = no_list[0]
    end_no = no_list[0]
    for no in no_list[1:]:
        if no == end_no + 1:
            end_no = no
        else:
            output.append(f"{start_no - 1}:{end_no - 1}")
            start_no = no
            end_no = no
    output.append(f"{start_no - 1}:{end_no - 1}")
    output_dict[f'"{category}"'] = output

# 원하는 형식으로 출력
for key, value in output_dict.items():
    print(f"{key} : {value}")


"곡류 및 서류" : ['49:62', '5374:5375', '6759:6767', '6771:6773', '7345:7345']
"과자류" : ['265:269', '1185:1185', '1187:1199', '1398:1402', '1621:1622', '2170:2173', '2276:2276', '2283:2283', '2285:2285', '2300:2305', '2308:2309', '2312:2316', '2346:2348', '2358:2359', '2378:2378', '2380:2380', '2583:2588', '2595:2600', '3200:3200', '3766:3775', '3777:3783', '3786:3791', '3793:3794', '4047:4047', '4059:4062', '4066:4066', '4070:4070', '4091:4091', '4095:4096', '4394:4400', '4402:4404', '4673:4673', '4785:4785', '4795:4796', '5047:5047', '5049:5049', '5093:5093', '5095:5095', '5098:5098', '5105:5105', '5107:5107', '5115:5115', '5169:5169', '5172:5174', '5178:5178', '5186:5188', '5190:5192', '5194:5207', '5209:5213', '5256:5256', '5535:5537', '5539:5539', '6143:6145', '6228:6228', '6270:6271', '6278:6279', '6302:6302', '6359:6364', '6475:6487', '6493:6502', '6506:6508', '6511:6512', '6522:6524', '6598:6598', '6614:6615', '6728:6733', '6736:6736', '6758:6758']
"구이류" : ['0:14', '453:453', '771:77

In [83]:
def get_food_prefs(food_log_categories):

  wweia_food_cats_list = wweia_food_categories['NO'].tolist()

  # Make dict to hold the number of foods that belong to a
  # specific WWEIA category. Key is the broad food category.

  cats_dict = {}
  cats_dict["곡류 및 서류"] = defaultdict(int)
  cats_dict["과자류"] = defaultdict(int)
  cats_dict["구이류"] = defaultdict(int)
  cats_dict["국 및 탕류"] = defaultdict(int)
  cats_dict["기타"] = defaultdict(int)
  cats_dict["김치류"] = defaultdict(int)
  cats_dict["농축산물"] = defaultdict(int)
  cats_dict["면 및 만두류"] = defaultdict(int)
  cats_dict["밥류"] = defaultdict(int)
  cats_dict["볶음류"] = defaultdict(int)
  cats_dict["빵류"] = defaultdict(int)
  cats_dict["생채및 무침류"] = defaultdict(int)
  cats_dict["수산물"] = defaultdict(int)
  cats_dict["숙채류"] = defaultdict(int)
  cats_dict["아이스크림류"] = defaultdict(int)
  cats_dict["음료 및 차류"] = defaultdict(int)
  cats_dict["장아찌 및 절임류"] = defaultdict(int)
  cats_dict["전.적 및 부침류"] = defaultdict(int)
  cats_dict["젓갈류"] = defaultdict(int)
  cats_dict["조림류"] = defaultdict(int)
  cats_dict["죽 및 스프류"] = defaultdict(int)
  cats_dict["찌개 및 전골류"] = defaultdict(int)
  cats_dict["찜류"] = defaultdict(int)
  cats_dict["튀김류"] = defaultdict(int)
  cats_dict["포류"] = defaultdict(int)
  cats_dict["회류"] = defaultdict(int)
  
  # Dict with lists containing cat codes for each broad food category
  
  category_mapping = { 
"곡류 및 서류" : wweia_food_cats_list[0:28],
"과자류" : wweia_food_cats_list[29:243],
"구이류" : wweia_food_cats_list[244:315],
"국 및 탕류" : wweia_food_cats_list[316:431],
"기타" : wweia_food_cats_list[432:516],
"김치류" : wweia_food_cats_list[517:542],
"농축산물" : wweia_food_cats_list[543:3812],
"면 및 만두류" : wweia_food_cats_list[3813:3939],
"밥류" : wweia_food_cats_list[3940:4049],
"볶음류" : wweia_food_cats_list[4050:4181],
"빵류" : wweia_food_cats_list[4182:7348],
"생채및 무침류" : wweia_food_cats_list[7349:7501],
"수산물" : wweia_food_cats_list[7502:8867],
"숙채류" : wweia_food_cats_list[8868:8890],
"아이스크림류" : wweia_food_cats_list[8891:9217],
"음료 및 차류" : wweia_food_cats_list[9218:11715],
"장아찌 및 절임류" : wweia_food_cats_list[11716:11739],
"전.적 및 부침류" : wweia_food_cats_list[11740:11787],
"젓갈류" : wweia_food_cats_list[11788:11794],
"조림류" : wweia_food_cats_list[11795:11836],
"죽 및 스프류" : wweia_food_cats_list[11837:11861],
"찌개 및 전골류" : wweia_food_cats_list[11862:11944],
"찜류" : wweia_food_cats_list[11945:11987],
"튀김류" : wweia_food_cats_list[11988:12314],
"포류" : wweia_food_cats_list[12315:12315],
"회류" : wweia_food_cats_list[12316:12318]}

  broad_categories_list = ["곡류 및 서류", "과자류", "구이류", "국 및 탕류", "기타", "김치류", "농축산물", "면 및 만두류", "밥류", "볶음류", "빵류", "생채및 무침류", "수산물", "숙채류", "아이스크림류", "음료 및 차류", "장아찌 및 절임류", "전.적 및 부침류", "젓갈류", "조림류", "죽 및 스프류", "찌개 및 전골류", "찜류", "튀김류", "포류", "회류"]

  for category_code in food_log_categories:

    if math.isnan(category_code): 
      continue # food does not have valid category, i.e. antacid

    # Loop through each category code and add 1 to its dict entry
    # if it's the right one
    for broad_cat in broad_categories_list:
      if category_code in category_mapping[broad_cat]:
        cats_dict[broad_cat][category_code] += 1

  max_code_dict = {}
  
  for broad_cat in broad_categories_list:
    

    if len(list(cats_dict[broad_cat].keys())) > 0:
      #print(cats_dict[broad_cat].keys())
      #print("zz: "+ broad_cat)
      max_code_dict[broad_cat] = max(cats_dict[broad_cat], key=cats_dict[broad_cat].get)

    else:
      max_code_dict[broad_cat] = "None of this category was eaten"

  return max_code_dict

In [84]:
def make_food_prefs_table(true_food_prefs, pred_food_prefs, wweia_synonym_cats):
  
  acc = 0
  sym_acc = 0

  for key in true_food_prefs:
    if true_food_prefs[key] == pred_food_prefs[key]:
      acc += 1
      sym_acc += 1
    else:
      if true_food_prefs[key] not in wweia_synonym_cats: continue
      elif pred_food_prefs[key] in wweia_synonym_cats[true_food_prefs[key]]:
        sym_acc += 1

  acc = acc / len(list(true_food_prefs.keys()))
  sym_acc = sym_acc / len(list(true_food_prefs.keys()))

  return acc, sym_acc

In [85]:
def get_pref_metrics(curr_log, n):
  
  true_pop_cats = get_most_pop_categories(curr_log, n, 'wweia_food_category_code')
  pref_pop_cats = get_most_pop_categories(curr_log, n, 'predicted_categories_number')

  true_pop_cats_sym = set()
  pref_pop_cats_sym = set()

  for elem in true_pop_cats:
    if elem in wweia_synonym_cats:
      for sym_elem in wweia_synonym_cats[elem]:
        true_pop_cats_sym.add(sym_elem)

  for elem in pref_pop_cats:
    if elem in wweia_synonym_cats:
      for sym_elem in wweia_synonym_cats[elem]:
        pref_pop_cats_sym.add(sym_elem)

  cats_common = set(true_pop_cats).intersection(set(pref_pop_cats))
  percent_common = len(cats_common) / n

  cats_common_sym = set(true_pop_cats_sym).intersection(set(pref_pop_cats_sym))
  percent_common_sym = min(1, len(cats_common_sym) / n)

  return percent_common, percent_common_sym

In [86]:
def food_recommandation_test(input_food_list) :
    input_list= ["NO", "Food Name", "food_category"]
    df = pd.DataFrame(input_food_list, columns=input_list)
    print(df)
    return df

In [87]:
two_dimensional_array = [
    [1, "라면", "면류"],
    [2, "스파게티", "면류"],
    [3, "생선", "구이류"]
]
food_recommandation_test(two_dimensional_array)

   NO Food Name food_category
0   1        라면            면류
1   2      스파게티            면류
2   3        생선           구이류


,NO,Food Name,food_category
0,1,라면,면류
1,2,스파게티,면류
2,3,생선,구이류


In [88]:
# Main method
def food_recommandation(input_food_list, wweia_synonym_cats) :
  avg_pref_acc = 0.0
  avg_pref_sym_acc = 0.0
  avg_percent_common = 0.0
  avg_percent_common_sym = 0.0
  avg_mean_rr = 0.0
  avg_sym_mean_rr = 0.0
  
  food_log_names = []
  pref_accs = []
  pref_sym_accs = []
  percent_common_list = []
  percent_common_sym_list = []
  mean_rrs = []
  sym_mean_rrs = []
  # NO, 식품명, 식품대분류
  #curr_log = pd.read_csv(path)

  input_list= ["wweia_food_category_code", "Food Name", "wweia_food_category_description"]
  curr_log = pd.DataFrame(input_food_list, columns=input_list)
  
  curr_log = process_food_log(curr_log, wweia_synonym_cats)

  food_log_name = 'sample_food_log'
    # # Write to file saved in Google Drive folder
    # file_name = PATH_NAME + 'lfp_method4/' + food_log_name
    # curr_log.to_csv(file_name)
  
    # Compare to true food categories
  
    # Count most popular categories and get food preferences
  percent_common, percent_common_sym = get_pref_metrics(curr_log, 10)
  print("Percent Top N Categories Identified:", percent_common, "Percent Top N Synonymous Categories Identified:", percent_common_sym)
  print("weii 출력 ")
  print(set(curr_log.loc[:,'wweia_food_category_code'].tolist()))
  
  print("다음 출력 ")
  print(set(curr_log.loc[:,'predicted_categories_number'].tolist()))
  # true_food_prefs = get_food_prefs(set(curr_log.loc[:,'wweia_food_category_code'].tolist()))
  # pred_food_prefs = get_food_prefs(set(curr_log.loc[:,'predicted_categories_number'].tolist()))
  
  # print(true_food_prefs)
  # print(pred_food_prefs)

  # pref_acc, pref_sym_acc = make_food_prefs_table(true_food_prefs, pred_food_prefs, wweia_synonym_cats)
  # print("Preference Accuracy:", pref_acc, "Preference Synonymous Accuracy:", pref_sym_acc)
  
  # mean_rr = np.mean(curr_log.loc[:,'reciprocal_rank'])
  # sym_mean_rr = np.mean(curr_log.loc[:,'sym_reciprocal_rank'])
  # avg_mean_rr += mean_rr
  # avg_sym_mean_rr += sym_mean_rr
  # mean_rrs.append(mean_rr)
  # sym_mean_rrs.append(sym_mean_rr)
  # print("Mean Reciprocal Rank:", mean_rr, "Synonymous Mean Reciprocal Rank:", sym_mean_rr)
  
  
  # Averaged statistics for all food logs
  # avg_acc /= len(os.listdir(path))
  # avg_sym_acc /= len(os.listdir(path))
  # avg_pref_acc /= len(os.listdir(path))
  # avg_pref_sym_acc /= len(os.listdir(path))
  # avg_percent_common /= len(os.listdir(path))
  # avg_percent_common_sym /= len(os.listdir(path))
  # avg_mean_rr /= len(os.listdir(path))
  # avg_sym_mean_rr /= len(os.listdir(path))
  # print("Average Accuracy:", avg_acc, "Average Synonymous Accuracy:", avg_sym_acc)
  # print("Average Mean Reciprocal Rank:", avg_mean_rr, "Average Synonymous Mean Reciprocal Rank:", avg_sym_mean_rr)
  # print("Average Preference Accuracy:", avg_pref_acc, "Average Synonymous Preference Accuracy:", avg_pref_sym_acc)
  # print("Average Percent Top N Categories Identified:", avg_percent_common, "Average Percent Top N Synonymous Categories Identified:", avg_percent_common_sym)
  
  
  

In [89]:
import pandas as pd
import random

# CSV 파일 읽기
df = pd.read_csv(PATH_NAME+ "wweia_food_categories_addtl.csv")

# 100개의 랜덤한 번호 선택
random_numbers = random.sample(range(1, 7684), 100)

# 선택된 번호에 해당하는 데이터 가져오기
selected_data = df[df['NO'].isin(random_numbers)]

# 필요한 열만 선택하여 리스트로 변환
food_list = selected_data[['NO', '식품명', '식품대분류']].values.tolist()

# 결과 출력
print(food_list)


[[47, '열무얼갈이김치', '김치류'], [53, '녹두시루떡', '곡류 및 서류'], [164, '치즈피자', '빵류'], [170, '간장마늘쫑장아찌', '장아찌 및 절임류'], [202, '조기조림', '조림류'], [247, '전복탕', '국 및 탕류'], [279, '글램핑 바비큐 더블치즈엣지(오리지널 L)', '빵류'], [311, '베스트 콰트로 더블치즈엣지(오리지널 M)', '빵류'], [316, '베이컨체더치즈(씬 L)', '빵류'], [667, '스윗피스 크림치즈R', '빵류'], [726, '찰떡단고감 노엣지L', '빵류'], [872, '전주불백피자R', '빵류'], [1034, '바닐라 빈', '아이스크림류'], [1089, '아이스 카푸치노', '음료 및 차류'], [1172, '딸기젤빙', '아이스크림류'], [1253, '아이스 카푸치노S', '음료 및 차류'], [1406, '플레인 카스테라', '빵류'], [1529, '바밤 라떼(HOT)', '음료 및 차류'], [1557, '그린티', '음료 및 차류'], [1560, '허브티(민트)', '음료 및 차류'], [1713, '시나몬 카페모카 tall (hot)', '음료 및 차류'], [1819, '그린 티 탐앤치노 grande', '음료 및 차류'], [1840, '프리미엄 스무디 블루베리 grande', '음료 및 차류'], [1881, '아이스블루스노우모카 R', '음료 및 차류'], [1908, '아이스 화이트모카 L', '음료 및 차류'], [1953, '아이스 토피넛 라떼 L', '음료 및 차류'], [1955, '아이스 고구마라떼 L', '음료 및 차류'], [2052, '오리지널 깔라만시 R', '음료 및 차류'], [2168, '모카번', '빵류'], [2193, '아이스 흑당 곡물라떼', '음료 및 차류'], [2326, '에그 브리오슈', '빵류'], [2460, '카페모카R', '음료 및 차류'], [2483, '까사링고 티라미수', '빵류'], [25

In [90]:
two_dimensional_array = [
    [1, "메밀전병", "곡류 및 서류"],
    [2, "약식", "곡류 및 서류"],
    [3, "무지개떡", "곡류 및 서류"],
    [45, "초코 마카롱", "과자류"],
    [106, "레몬머랭", "과자류"],
    [105, "코코넛머랭", "과자류"],
    [335, "홍합미역국", "국 및 탕류"]
]
food_recommandation(food_list, wweia_synonym_cats)

C:\Users\woobi\AppData\Local\Temp\ipykernel_15324\1859825223.py:90: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0. 0. 0. 0. 0.]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  curr_log.loc[i, 'max_cosim_score'] = np.array2string(to_keep[0,-5:])


Percent Top N Categories Identified: 0.0 Percent Top N Synonymous Categories Identified: 1
weii 출력 
{2052, 5636, 1034, 5645, 6674, 1557, 4119, 1560, 3625, 47, 2554, 53, 5181, 1089, 4681, 7256, 7262, 5739, 3693, 6260, 4726, 5239, 2168, 3711, 3716, 6283, 2193, 1172, 7317, 4248, 4760, 667, 6811, 164, 170, 1713, 7354, 6338, 202, 726, 3286, 5338, 6883, 1253, 4845, 7406, 3314, 247, 7418, 5893, 3849, 5898, 6419, 2326, 279, 3351, 1819, 5917, 4392, 6953, 1840, 3891, 311, 3899, 316, 4413, 5968, 5459, 1881, 2906, 7516, 872, 1908, 1406, 3967, 4486, 2460, 4510, 1953, 1955, 7597, 2483, 7091, 4535, 4028, 5571, 7619, 4552, 3530, 3022, 4050, 3541, 7129, 3039, 5604, 7143, 5612, 1529, 4090, 4095}
다음 출력 
{256, 3, 5, 6, 139, 146, 275, 20, 405, 148, 23, 663, 25, 276, 1045, 28, 1820, 290, 34, 43, 2095, 48, 176, 50, 52, 439, 57, 314, 58, 2746, 190, 2116, 330, 2766, 1623, 732, 353, 230, 105, 111, 1146, 124, 253, 3710}


In [63]:
# Make table to show accuracies
df = pd.DataFrame(list(zip(food_log_name, accs, sym_accs, pref_accs, pref_sym_accs, percent_common_list, percent_common_sym_list, mean_rrs, sym_mean_rrs)),
               columns =['Food Log', 'Accuracies', "Synomynous Accuracies", "Preference Accuracies", "Synomynous Preference Accuracies", "Percent of Top N Categories Achieved", "Percent of Top N Categories Achieved Sym", "Mean RR", "Sym Mean RR"])
df

NameError: name 'food_log_name' is not defined